In [14]:
from datetime import datetime
from zhdate import ZhDate

#声明变量
chinesedate = "二零二五年四月初九 乙巳年 (蛇年)"
date = 6
fulldate = "2025年5月6日 星期三"

num_week = 6
num_percentage = 67

week = "Week " + str(num_week)
percentage = str(num_percentage) + "%"
schedule = "测试\n测试\n测试"

#实现代码
today = ZhDate.today()
chinesedate = today.chinese()

with open("panel.html", "r", encoding="utf-8") as f:
    panel = f.read()

html = panel.replace("{{chinesedate}}", chinesedate)\
               .replace("{{date}}", str(date))\
               .replace("{{fulldate}}",fulldate)\
               .replace("{{week}}", week)\
               .replace("{{percentage}}", percentage)\
               .replace("{{schedule}}", schedule)

with open("calendar.html", "w", encoding="utf-8") as f:
    f.write(html)

In [49]:
import requests
from icalendar import Calendar
from datetime import datetime, date, timedelta
from dateutil.rrule import rrulestr
import pytz

# 替换 webcal 为 https
url = "https://p215-caldav.icloud.com.cn/published/2/MTY5NDM3NDIyODkxNjk0MxQGglZSrjcCCgbEAcXHCbH1PEeMJlXixNdWItQaJ3vTpLQG9mBKbwfAoA3nGulbAnNxSzx-Nh8QY1dQ9ZSBoTQ"

# 下载 ICS 文件
response = requests.get(url)
if response.status_code != 200:
    print("无法获取 ICS 文件，状态码:", response.status_code)
    exit()

# 解析 ICS 内容
cal = Calendar.from_ical(response.text)

# 本地今天的日期（你也可以改成 date(2025, 5, 13) 手动指定）
today = date.today()

events_today = []

# 遍历所有事件
for component in cal.walk():
    if component.name == "VEVENT":
        summary = component.get("summary")
        description = component.get("description")
        location = component.get("location")
        dtstart = component.get("dtstart").dt
        dtend = component.get("dtend").dt

        # 确定是否带时区
        tzinfo = dtstart.tzinfo if isinstance(dtstart, datetime) else None

        # 构建时区一致的 today 时间段
        start_of_today = datetime.combine(today, datetime.min.time())
        end_of_today = datetime.combine(today + timedelta(days=1), datetime.min.time())

        if tzinfo is not None:
            start_of_today = start_of_today.replace(tzinfo=tzinfo)
            end_of_today = end_of_today.replace(tzinfo=tzinfo)

        # 处理重复事件
        if component.get('rrule'):
            rule_str = component.get('rrule').to_ical().decode()
            rule = rrulestr(rule_str, dtstart=dtstart)

            occurrences = rule.between(start_of_today, end_of_today, inc=True)

            for occur in occurrences:
                events_today.append({
                    "title": summary,
                    "start": occur,
                    "end": occur + (dtend - dtstart),
                    "desc": description,
                    "location": location
                })
        else:
            # 非重复事件
            event_date = dtstart.date() if isinstance(dtstart, datetime) else dtstart
            if event_date == today:
                events_today.append({
                    "title": summary,
                    "start": dtstart,
                    "end": dtend,
                    "desc": description,
                    "location": location
                })

# 打印结果
output_lines = []

if events_today:
    events_today.sort(key=lambda e: e["start"])

    for event in events_today:
        start_time = event["start"].astimezone(pytz.timezone("Asia/Kuala_Lumpur")).strftime("%H:%M")
        end_time = event["end"].astimezone(pytz.timezone("Asia/Kuala_Lumpur")).strftime("%H:%M")
        line = f" ·{event['title']} {start_time}-{end_time}"
        output_lines.append(line)
        if event["desc"]:
            output_lines.append(f" （{event['desc']}）")
else:
    output_lines.append(f"今天（{today}）没有事件。")

# 最终字符串
output = "\n".join(output_lines)

# 可打印查看
print(output)



 ·课程1 06:00-07:00
 （A1-303）
 ·课程2 09:00-10:00
 （A2-303）


In [18]:
from datetime import datetime
from zhdate import ZhDate
import requests
from icalendar import Calendar
from datetime import datetime, date, timedelta
from dateutil.rrule import rrulestr
import pytz
from bs4 import BeautifulSoup

#声明变量
chinesedate = "二零二五年四月初九 乙巳年 (蛇年)"
day = 6
fulldate = "2025年5月6日 星期三"

num_week = 6
percentage = "67%"

schedule = "测试\n测试\n测试"
url = "https://p215-caldav.icloud.com.cn/published/2/MTY5NDM3NDIyODkxNjk0MxQGglZSrjcCCgbEAcXHCbH1PEeMJlXixNdWItQaJ3vTpLQG9mBKbwfAoA3nGulbAnNxSzx-Nh8QY1dQ9ZSBoTQ"
disp_time = datetime.now().strftime("%H:%M")

#实现代码
#---------------农历str------------
def getChineseDate():
    today = ZhDate.today()
    chinesedate = today.chinese()
    return chinesedate
    
#------------日程表----------------
def getSchedule():
    import requests
    from icalendar import Calendar
    from datetime import datetime, date, timedelta
    from dateutil.rrule import rrulestr

    url = "https://p215-caldav.icloud.com.cn/published/2/MTY5NDM3NDIyODkxNjk0MxQGglZSrjcCCgbEAcXHCbH1PEeMJlXixNdWItQaJ3vTpLQG9mBKbwfAoA3nGulbAnNxSzx-Nh8QY1dQ9ZSBoTQ"
    response = requests.get(url)
    if response.status_code != 200:
        return "（日程加载失败）"

    cal = Calendar.from_ical(response.text)
    today = date.today()

    all_day_events = []
    timed_events = []

    for component in cal.walk():
        if component.name != "VEVENT":
            continue

        summary = component.get("summary")
        description = component.get("description")
        dtstart = component.get("dtstart").dt
        dtend = component.get("dtend").dt

        # 判断是否是全天事件
        if isinstance(dtstart, date) and not isinstance(dtstart, datetime):
            if dtstart == today:
                all_day_events.append({
                    "title": summary,
                    "desc": description
                })
        else:
            # 时间事件处理
            tzinfo = dtstart.tzinfo
            start_of_today = datetime.combine(today, datetime.min.time()).replace(tzinfo=tzinfo)
            end_of_today = datetime.combine(today + timedelta(days=1), datetime.min.time()).replace(tzinfo=tzinfo)


            if component.get('rrule'):
                rule_str = component.get('rrule').to_ical().decode()
                rule = rrulestr(rule_str, dtstart=dtstart)
                occurrences = rule.between(start_of_today, end_of_today, inc=True)

                for occur in occurrences:
                    timed_events.append({
                        "title": summary,
                        "start": occur,
                        "end": occur + (dtend - dtstart),
                        "desc": description
                    })
            else:
                if dtstart.date() == today:
                    timed_events.append({
                        "title": summary,
                        "start": dtstart,
                        "end": dtend,
                        "desc": description
                    })

    # 构建输出字符串
    output_lines = []

    for event in all_day_events:
        output_lines.append(f" ·{event['title']} 全天")
        if event["desc"]:
            output_lines.append(f" （{event['desc']}）")

    if timed_events:
        timed_events.sort(key=lambda e: e["start"])
        for event in timed_events:
            start_time = event["start"].strftime("%H:%M")
            end_time = event["end"].strftime("%H:%M")
            output_lines.append(f" ·{event['title']} {start_time}-{end_time}")
            if event["desc"]:
                output_lines.append(f" （{event['desc']}）")

    if not all_day_events and not timed_events:
        output_lines.append(f"今天（{today}）没有事件。")

    return "\n".join(output_lines)

#-----------获取天数---------------
def getDay():
    today = date.today()
    day = today.day
    return day
    
#-----------获取完整日期-----------
def getFulldate():
    today = date(2025, 5, 6)  # 你也可以用 date.today()
    weekday_cn = "星期" + "一二三四五六日"[today.weekday()]
    fulldate = f"{today.year}年{today.month}月{today.day}日 {weekday_cn}"
    return fulldate
#-----------获取学期周-------------
def getWeek():
    semester_start = date(2025, 3, 28)

    # 今天日期（你也可以手动指定）
    today = date.today()

    # 计算周数（+1 表示第几周，从1开始）
    delta_days = (today - semester_start).days
    week_number = delta_days // 7 + 1
    return week_number

#---------获取当前时间降水概率-------
def getPercentage():
    url = "https://weather.com/zh-CN/weather/hourbyhour/l/Kuala%2BLumpur%2BMalaysia?canonicalCityId=91a6f9c5c0d51f40d6a3eb94a0498ed0"
    headers = {
    "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 只获取第一个降水概率
    first_precip = soup.find('span', attrs={"data-testid": "PercentageValue"})

    if first_precip:
        return first_precip.get_text()
    else:
        return"未找到"

#----------统一部署自定义函数--------
chinesedate = getChineseDate()

schedule = getSchedule()

day = getDay()

fulldate = getFulldate()

num_week = getWeek()

week = "Week " + str(num_week)

percentage = getPercentage()

#----------添加到html-------------

with open("panel.html", "r", encoding="utf-8") as f:
    panel = f.read()

html = panel.replace("{{chinesedate}}", chinesedate)\
               .replace("{{date}}", str(day))\
               .replace("{{fulldate}}",fulldate)\
               .replace("{{week}}", week)\
               .replace("{{percentage}}", percentage)\
               .replace("{{schedule}}", schedule)\
               .replace("{{time}}", disp_time) 

with open("calendar.html", "w", encoding="utf-8") as f:
    f.write(html)

二零二五年四月十六 乙巳年 (蛇年)
 ·全天课程测试 全天
 （A3-303）
 ·课程1 06:00-07:00
 （A1-303）
 ·课程2 09:00-10:00
 （A2-303）
13
2025年5月6日 星期二
Week 7
69%


In [12]:
import requests
from bs4 import BeautifulSoup

url = "https://weather.com/zh-CN/weather/hourbyhour/l/Kuala%2BLumpur%2BMalaysia?canonicalCityId=91a6f9c5c0d51f40d6a3eb94a0498ed0"
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# 找出所有降水概率
precips = soup.find_all('span', attrs={"data-testid": "PercentageValue"})

for p in precips:
    print("降水概率：", p.get_text())

降水概率： 69%
降水概率： 77%
降水概率： 100%
降水概率： 82%
降水概率： 82%
降水概率： 100%
降水概率： 79%
降水概率： 83%
降水概率： 100%
降水概率： 14%
降水概率： 81%
降水概率： 93%
降水概率： 11%
降水概率： 81%
降水概率： 95%
降水概率： 12%
降水概率： 84%
降水概率： 94%
降水概率： 21%
降水概率： 84%
降水概率： 91%
降水概率： 24%
降水概率： 86%
降水概率： 91%
降水概率： 20%
降水概率： 86%
降水概率： 94%
降水概率： 19%
降水概率： 87%
降水概率： 90%
降水概率： 24%
降水概率： 87%
降水概率： 74%
降水概率： 24%
降水概率： 88%
降水概率： 77%
降水概率： 23%
降水概率： 88%
降水概率： 82%
降水概率： 21%
降水概率： 82%
降水概率： 92%
降水概率： 16%
降水概率： 74%
降水概率： 90%
降水概率： 15%
降水概率： 67%
降水概率： 88%
降水概率： 15%
降水概率： 61%
降水概率： 90%
降水概率： 11%
降水概率： 57%
降水概率： 92%
降水概率： 11%
降水概率： 56%
降水概率： 83%
降水概率： 36%
降水概率： 58%
降水概率： 82%
降水概率： 46%
降水概率： 61%
降水概率： 75%
降水概率： 14%
降水概率： 65%
降水概率： 75%
降水概率： 16%
降水概率： 69%
降水概率： 77%
降水概率： 19%
降水概率： 73%
降水概率： 82%
降水概率： 21%
降水概率： 77%
降水概率： 72%
降水概率： 21%
降水概率： 79%
降水概率： 74%
降水概率： 24%
降水概率： 81%
降水概率： 75%
降水概率： 19%
降水概率： 82%
降水概率： 81%
降水概率： 11%
降水概率： 83%
降水概率： 89%
降水概率： 6%
降水概率： 84%
降水概率： 80%
降水概率： 8%
降水概率： 84%
降水概率： 55%
降水概率： 7%
降水概率： 85%
降水概率： 48%
降水概率： 24%
降水概率： 86%
降水概率： 32%
降水概率： 24%
